# <center> **502 Project** </center>
### <center> Team member: Chenxi Liu, Nuo Tian, Mengyu Liu, Yuan Liu </center>

### Data Selection: 
#### Yahoo News Data Set (Part 1 and 2of 35) 




### STEP 1 : Data Extracting and Processing 

In [14]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder \
    .appName("project")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=project>

#### 1. Build Schema Using the README Instruction of the Data Set 
#### 

In [15]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("article_type", StringType(), True),
    StructField("np1", StringType(), True),
    StructField("np2", StringType(), True),
    StructField("context", StringType(), True),
    StructField("source", StringType(), True),
    StructField("category", StringType(), True),
    StructField("location", StringType(), True),
    StructField("time", StringType(), True),])

#### 2. Readin the Data Set 
#### 

In [16]:
df = spark.read.csv("s3://anly502project/data/part-r-00000",sep = "\t",header=False,schema=schema)
df_2 = spark.read.csv("s3://anly502project/data/part-r-00001",sep = "\t",header=False,schema=schema)

Py4JJavaError: An error occurred while calling o256.csv.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:547)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:545)
	at scala.collection.TraversableLike$$anonfun$fl

#### 3. Verify the Schema 
#### 

In [24]:
#### Data Schema
df.printSchema()
df_2.printSchema()

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)



#### 4. Combine the Data Set
#### 

In [25]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [26]:
unioned_df = unionAll([df, df_2])


In [27]:
#### show combined
unioned_df.show(10)

+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|article_type|            np1|np2|             context|       source|            category|            location| time|
+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             |intlnews topstor ...|      , kerala india|14299|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|     article|    Communities|  E|arg1 mobilised in...|             |    politics topstor|                   ,|14026|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|            |     Coast bias|  E|arg2 is for East ...|             |      sports topstor| columbus, ohio u...|13956|
|     article|Commerce office|  E|arg1 at DDD Linco...| 

In [28]:
#### show number of the rows 
unioned_df.count()

196017242

In [29]:
####Drop unrelated Column
unioned_df = unioned_df.drop("source").drop("article_type").drop("location").drop("time")

In [30]:
####Show data frame after dropping unrelated columns 
unioned_df.show(10)

+---------------+---+--------------------+--------------------+
|            np1|np2|             context|            category|
+---------------+---+--------------------+--------------------+
|    Dark Knight|  E|  arg1 and Wall arg2|intlnews topstor ...|
|    Carotenoids|  E|arg1 and caroteno...|topstor,health,sc...|
|    Communities|  E|arg1 mobilised in...|    politics topstor|
|    Carotenoids|  E|arg1 and caroteno...|topstor,health,sc...|
|     Coast bias|  E|arg2 is for East ...|      sports topstor|
|Commerce office|  E|arg1 at DDD Linco...|topstor,lifestle,...|
| 75-minute mark|  E|arg1 in a pulsati...|      topstor,sports|
| Brigham Circle|  E|      arg2 past arg1|             topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews headlin...|
| Brigham Circle|  E|arg2 line service...|             topstor|
+---------------+---+--------------------+--------------------+
only showing top 10 rows



#### 5. Select the very top category by using multiple split 
#### 

In [31]:
from pyspark.sql.functions import split
split_col = split(unioned_df['category'], ',')
unioned_df = unioned_df.withColumn('category', split_col.getItem(0))

In [32]:
unioned_df.show(10)

+---------------+---+--------------------+--------------------+
|            np1|np2|             context|            category|
+---------------+---+--------------------+--------------------+
|    Dark Knight|  E|  arg1 and Wall arg2|intlnews topstor ...|
|    Carotenoids|  E|arg1 and caroteno...|             topstor|
|    Communities|  E|arg1 mobilised in...|    politics topstor|
|    Carotenoids|  E|arg1 and caroteno...|             topstor|
|     Coast bias|  E|arg2 is for East ...|      sports topstor|
|Commerce office|  E|arg1 at DDD Linco...|             topstor|
| 75-minute mark|  E|arg1 in a pulsati...|             topstor|
| Brigham Circle|  E|      arg2 past arg1|             topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews headlin...|
| Brigham Circle|  E|arg2 line service...|             topstor|
+---------------+---+--------------------+--------------------+
only showing top 10 rows



In [33]:
split_col_2 = split(unioned_df['category'], ' ')
unioned_df = unioned_df.withColumn('category', split_col_2.getItem(0))

In [34]:
split_col_3 = split(unioned_df['category'], '_')
unioned_df = unioned_df.withColumn('category', split_col_3.getItem(0))

In [35]:
split_col_4 = split(unioned_df['category'], '-')
unioned_df = unioned_df.withColumn('category', split_col_4.getItem(0))

In [36]:
#### Show data frame after filtering the category
unioned_df.show(10)

+---------------+---+--------------------+---------+
|            np1|np2|             context| category|
+---------------+---+--------------------+---------+
|    Dark Knight|  E|  arg1 and Wall arg2| intlnews|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|    Communities|  E|arg1 mobilised in...| politics|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|     Coast bias|  E|arg2 is for East ...|   sports|
|Commerce office|  E|arg1 at DDD Linco...|  topstor|
| 75-minute mark|  E|arg1 in a pulsati...|  topstor|
| Brigham Circle|  E|      arg2 past arg1|  topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews|
| Brigham Circle|  E|arg2 line service...|  topstor|
+---------------+---+--------------------+---------+
only showing top 10 rows



#### 6. Category (label) insight 
#### 

In [37]:
#### Get some insight of the label 
unioned_df.groupBy("category").count().show()

+------------+-------+
|    category|  count|
+------------+-------+
|      ciridi|     70|
|      travel|    890|
|      cirmus|     76|
|      cirngc|      3|
|      cirrfs|      2|
|regionalnews|1280737|
|      launch|   3652|
|      cirhge|   2520|
|      cirmec|    171|
|      cje2ds|     51|
|      cje2de|    223|
|      cirsvu|      3|
|      cirhw6|   3173|
|       cirgw|    367|
|      cirgha|    279|
|      cje3fq|     54|
|      cje2fa|     43|
|      cirlwu|      2|
|      cirida|  14761|
|      cirfvc|   1809|
+------------+-------+
only showing top 20 rows



In [38]:
#### Filter out the null and empty category 
unioned_df = unioned_df.filter(unioned_df.category.isNotNull())
unioned_df = unioned_df.filter(unioned_df.category != '')

In [39]:
#### More insight 
from pyspark.sql.functions import countDistinct
unioned_df.agg(countDistinct(unioned_df.category).alias('distinct_label')).collect()
##### We can see we have around 1116 distinct labels which some of it does not make any sense so we dicided to filter it out 

KeyboardInterrupt: 

#### 7. Using the SQL to filter the category (label)
#### 

In [1]:

import pyspark.sql as sql
count_df.createOrReplaceTempView("count_df")
count_rank_df = spark.sql("SELECT category, count FROM count_df ORDER BY count DESC LIMIT 15")

NameError: name 'count_df' is not defined

In [47]:
count_rank_df.show(15)

+-------------+--------+
|     category|   count|
+-------------+--------+
|      topstor|85027590|
|    localnews|40174779|
|       sports|22925153|
|     business|20017107|
| nationalnews| 5073191|
|     intlnews| 4252106|
|    technolog| 2546612|
|entertainment| 2359067|
|     politics| 2092784|
|     lifestle| 1785076|
| regionalnews| 1280737|
|       health|  980067|
|        world|  705658|
|           gf|  558948|
|      science|  533434|
+-------------+--------+



In [49]:
unioned_df.createOrReplaceTempView("unioned_df")
count_rank_df.createOrReplaceTempView("count_rank_df")
df_final = spark.sql("SELECT * FROM unioned_df WHERE unioned_df.category IN (SELECT category FROM count_rank_df)")

In [50]:
df_final.show(10)

+---------------+---+--------------------+---------+
|            np1|np2|             context| category|
+---------------+---+--------------------+---------+
|    Dark Knight|  E|  arg1 and Wall arg2| intlnews|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|    Communities|  E|arg1 mobilised in...| politics|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|     Coast bias|  E|arg2 is for East ...|   sports|
|Commerce office|  E|arg1 at DDD Linco...|  topstor|
| 75-minute mark|  E|arg1 in a pulsati...|  topstor|
| Brigham Circle|  E|      arg2 past arg1|  topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews|
| Brigham Circle|  E|arg2 line service...|  topstor|
+---------------+---+--------------------+---------+
only showing top 10 rows



In [51]:
df_final.agg(countDistinct(df_final.category).alias('distinct_label')).collect()

[Row(distinct_label=15)]

In [52]:
#### Verfied the final data frame 
df_final.groupBy("category").count().show()

+-------------+--------+
|     category|   count|
+-------------+--------+
| regionalnews| 1280737|
|      topstor|85027590|
|     lifestle| 1785076|
|           gf|  558948|
|     politics| 2092784|
|      science|  533434|
|       health|  980067|
|    technolog| 2546612|
|       sports|22925153|
|        world|  705658|
|entertainment| 2359067|
| nationalnews| 5073191|
|     business|20017107|
|    localnews|40174779|
|     intlnews| 4252106|
+-------------+--------+



In [4]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('words')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [6]:
documentAssembler = DocumentAssembler() \
    .setInputCol('context') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

In [7]:
import sparknlp
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

spark = sparknlp.start()
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.5
Apache Spark version:  2.4.5


In [52]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en') \
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)\
    .setPoolingLayer(0) # default 0

bert_base_cased download started this may take some time.


TypeError: 'JavaPackage' object is not callable

In [12]:
sc.stop()

In [13]:
SparkSession._instantiatedContext = None